# Query Benchmark
This benchmark demostrates the efficiency of using columnar data formats. Here we run 4 benchmarks on the complete PDB to Uniprot residue-level mapping with a total of 105,594,955 records as of July 28, 2018.

1. Count number of records
2. Run a query
3. Join datasets
4. Convert to a Pandas dataframe

In [1]:
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88") # some pandas - numpy version issue

In [2]:
%%time
spark = SparkSession.builder.master("local[4]").appName("Example").getOrCreate()
spark.conf.set("spark.sql.orc.impl", "native")

CPU times: user 25.3 ms, sys: 16.1 ms, total: 41.4 ms
Wall time: 3.6 s


# 1. Count number of records
Read PDB to UniProt mapping file in the ORC columnar data format.

In [3]:
%%time
# the dataset in ORC format
ds = spark.read.orc("./data/pdb2uniprot_residues.orc.lzo")
# the dataset in Parquet format
#ds = spark.read.parquet("./data/pdb2uniprot_residues.parquet.gzip")

print("total number of records: " + str(ds.count()))

total number of records: 105594955
CPU times: user 3.26 ms, sys: 2.07 ms, total: 5.33 ms
Wall time: 4.1 s


# 2. Run a query
## Find Mitogen-activated protein kinase 14
Here we run a query for PDB - UniProt mappings for UniProt ID Q16539 (MK14_HUMAN) and retrieve their residue-level mappings for residues that are observed in the PDB structure (e.g., pdbResNum is not null).

In [4]:
%%time
mk14_human = ds.filter("uniprotId == 'Q16539' AND pdbResNum != 'null'")

print("Number of distinct chains: " + str(mk14_human.select("structureChainId").distinct().count()))

print("Number of residue mappings: " + str(mk14_human.count()))
mk14_human.show()

Number of distinct chains: 243
Number of residue mappings: 82277
+----------------+---------+---------+---------+----------+
|structureChainId|pdbResNum|pdbSeqNum|uniprotId|uniprotNum|
+----------------+---------+---------+---------+----------+
|          2ZB1.A|        4|        4|   Q16539|         4|
|          2ZB1.A|        5|        5|   Q16539|         5|
|          2ZB1.A|        6|        6|   Q16539|         6|
|          2ZB1.A|        7|        7|   Q16539|         7|
|          2ZB1.A|        8|        8|   Q16539|         8|
|          2ZB1.A|        9|        9|   Q16539|         9|
|          2ZB1.A|       10|       10|   Q16539|        10|
|          2ZB1.A|       11|       11|   Q16539|        11|
|          2ZB1.A|       12|       12|   Q16539|        12|
|          2ZB1.A|       13|       13|   Q16539|        13|
|          2ZB1.A|       14|       14|   Q16539|        14|
|          2ZB1.A|       15|       15|   Q16539|        15|
|          2ZB1.A|       16|       

# 3. Join operation

In [5]:
# create a random dataset of ~10,000 chains
sample = ds.sample(withReplacement=False, fraction=0.0001, seed=123).select("structureChainId").withColumnRenamed("structureChainId", "id").distinct()

print("Sample size: " + str(sample.count()))
sample.show()

Sample size: 10268
+-------+
|     id|
+-------+
| 5JPQ.h|
| 3AL5.C|
| 4MY9.D|
| 4FZB.K|
| 1YT5.A|
| 1E5Q.A|
| 5TU9.A|
| 2VKZ.A|
| 1YNJ.J|
| 2M89.B|
| 5LUF.V|
| 5TAN.E|
|4U4N.D9|
|4U4N.n5|
| 5WZT.A|
| 5JJ1.J|
| 1JRK.A|
| 5UTS.G|
| 3INN.C|
| 4FC4.E|
+-------+
only showing top 20 rows



Now we use this sample dataset to run a database inner join for ~10,000 records

In [6]:
%%time
subset = ds.join(sample, ds.structureChainId == sample.id).drop(sample.id)

print("Number of residue in subset: " + str(subset.count()))
subset.show()

Number of residue in subset: 4192893
+----------------+---------+---------+---------+----------+
|structureChainId|pdbResNum|pdbSeqNum|uniprotId|uniprotNum|
+----------------+---------+---------+---------+----------+
|          1F1E.A|     null|        1|   O93641|         1|
|          1F1E.A|     null|        2|   O93641|         2|
|          1F1E.A|     null|        3|   O93641|         3|
|          1F1E.A|        4|        4|   O93641|         4|
|          1F1E.A|        5|        5|   O93641|         5|
|          1F1E.A|        6|        6|   O93641|         6|
|          1F1E.A|        7|        7|   O93641|         7|
|          1F1E.A|        8|        8|   O93641|         8|
|          1F1E.A|        9|        9|   O93641|         9|
|          1F1E.A|       10|       10|   O93641|        10|
|          1F1E.A|       11|       11|   O93641|        11|
|          1F1E.A|       12|       12|   O93641|        12|
|          1F1E.A|       13|       13|   O93641|        13|
|  

# 4. Convert from Spark to Pandas dataframe

In [7]:
%%time
mk14_pd = mk14_human.toPandas()

CPU times: user 639 ms, sys: 75.1 ms, total: 714 ms
Wall time: 6.01 s


In [8]:
mk14_pd.head(20)

,structureChainId,pdbResNum,pdbSeqNum,uniprotId,uniprotNum
0,2ZB1.A,4,4,Q16539,4
1,2ZB1.A,5,5,Q16539,5
2,2ZB1.A,6,6,Q16539,6
3,2ZB1.A,7,7,Q16539,7
4,2ZB1.A,8,8,Q16539,8
5,2ZB1.A,9,9,Q16539,9
6,2ZB1.A,10,10,Q16539,10
7,2ZB1.A,11,11,Q16539,11
8,2ZB1.A,12,12,Q16539,12
9,2ZB1.A,13,13,Q16539,13


In [9]:
spark.stop()